In [3]:
import requests
from bs4 import BeautifulSoup
import re
import folium
import json
import pandas as pd
import numpy as np
import re

In [4]:
# Kakao API : 구주소 -> 신주소, 위경도 
def getLatLng(addr):
    with open('./account_id.json', 'r') as f:
        data = f.read()
        json_data = json.loads(data) # str -> obj
        app_id = json_data['kakao']['app_id']
        app_key = json_data['kakao']['app_key'] 
    
    #addr = '부평구 부평1동 534-48'
    app_key = 'KakaoAK ' + app_key
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+addr
    headers = {'Authorization': app_key}
    response = requests.get(url, headers=headers)
    result_json = response.json()
    if result_json['meta']['total_count'] > 0:
        x = result_json['documents'][0]['address']['x']
        y = result_json['documents'][0]['address']['y']
    else:
        x = 0
        y = 0
    return (y,x)

In [5]:
with open('./account_id.json', 'r') as f:
    data = f.read()
    json_data = json.loads(data) # str -> obj
    app_id = json_data['kakao']['app_id']
    app_key = json_data['kakao']['app_key'] 

addr = '제주특별자치도 제주시 한림읍 명월성로 16'
app_key = 'KakaoAK ' + app_key
url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+addr
headers = {'Authorization': app_key}
response = requests.get(url, headers=headers)
result_json = response.json()

In [6]:
result_json

{'documents': [{'address': {'address_name': '제주특별자치도 제주시 한림읍 동명리 1576-7',
    'b_code': '5011025026',
    'h_code': '5011025000',
    'main_address_no': '1576',
    'mountain_yn': 'N',
    'region_1depth_name': '제주특별자치도',
    'region_2depth_name': '제주시',
    'region_3depth_h_name': '한림읍',
    'region_3depth_name': '한림읍 동명리',
    'sub_address_no': '7',
    'x': '126.26554032813',
    'y': '33.4073746848732'},
   'address_name': '제주특별자치도 제주시 한림읍 명월성로 16',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '제주특별자치도 제주시 한림읍 명월성로 16',
    'building_name': '',
    'main_building_no': '16',
    'region_1depth_name': '제주특별자치도',
    'region_2depth_name': '제주시',
    'region_3depth_name': '한림읍 동명리',
    'road_name': '명월성로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.26554032813',
    'y': '33.4073746848732',
    'zone_no': '63013'},
   'x': '126.26554032813',
   'y': '33.4073746848732'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [7]:
# 가맹점 정보
jeju_merchant = pd.read_csv(', encoding = 'cp949')
jeju_merchant.head()

,YR,MCT_NM,MCT_BRN,LG_CAT,LG_CAT_NM,MD_CAT,MD_CAT_NM,MCT_BSE_AR,MNTH_SALES_PCTL,DAW_CCD,APV_TMT_CD,AGE_GP,SEX_CCD,UE_CT
0,2021,젠,2110164178,1,요식/유흥,2,일식/중식/양식,제주 서귀포시 안덕면 사계남로 186-8,3_25%~50%,1,4_09_12,4_35_39,F,42
1,2021,젠,2110164178,1,요식/유흥,2,일식/중식/양식,제주 서귀포시 안덕면 사계남로 186-8,3_25%~50%,2,6_15_18,2_25_29,F,42
2,2022,젠,2110164178,1,요식/유흥,2,일식/중식/양식,제주 서귀포시 안덕면 사계남로 186-8,3_25%~50%,3,7_18_21,3_30_34,M,42
3,2022,젠,2110164178,1,요식/유흥,2,일식/중식/양식,제주 서귀포시 안덕면 사계남로 186-8,3_25%~50%,5,7_18_21,3_30_34,F,42
4,2021,젠,2110164178,1,요식/유흥,2,일식/중식/양식,제주 서귀포시 안덕면 사계남로 186-8,3_25%~50%,6,6_15_18,2_25_29,M,42


In [8]:
# 필요한 가맹정 데이터 추출 및 중복 제거(가맹점명)
unique_df = jeju_merchant[['MCT_NM', 'LG_CAT', 'MD_CAT_NM', 'MD_CAT', 'MCT_BSE_AR', 'MNTH_SALES_PCTL']]
unique_df = unique_df.drop_duplicates(subset='MCT_NM', keep='first', inplace=False, ignore_index=False)
unique_df.head()

,MCT_NM,LG_CAT,MD_CAT_NM,MD_CAT,MCT_BSE_AR,MNTH_SALES_PCTL
0,젠,1,일식/중식/양식,2,제주 서귀포시 안덕면 사계남로 186-8,3_25%~50%
8,3657,1,제과/커피/패스트푸드,3,제주 서귀포시 안덕면 난드르로 56,4_50%~75%
9,가치,1,일식/중식/양식,2,제주특별자치도 제주시 동한두기길 17,4_50%~75%
11,곰막,1,한식,1,제주특별자치도 제주시 구좌읍 구좌해안로 64,3_25%~50%
18,너븐,1,일식/중식/양식,2,제주특별자치도 서귀포시 칠십리로214번길 26,4_50%~75%


In [9]:
unique_df[unique_df['LG_CAT'] == 1]

,MCT_NM,LG_CAT,MD_CAT_NM,MD_CAT,MCT_BSE_AR,MNTH_SALES_PCTL
0,젠,1,일식/중식/양식,2,제주 서귀포시 안덕면 사계남로 186-8,3_25%~50%
8,3657,1,제과/커피/패스트푸드,3,제주 서귀포시 안덕면 난드르로 56,4_50%~75%
9,가치,1,일식/중식/양식,2,제주특별자치도 제주시 동한두기길 17,4_50%~75%
11,곰막,1,한식,1,제주특별자치도 제주시 구좌읍 구좌해안로 64,3_25%~50%
18,너븐,1,일식/중식/양식,2,제주특별자치도 서귀포시 칠십리로214번길 26,4_50%~75%
...,...,...,...,...,...,...
221311,텐텐,1,유흥,5,제주특별자치도 제주시 연동8길 30,5_75%~90%
232003,미소,1,유흥,5,제주특별자치도 제주시 연동8길 7,6_90%미만
242844,하루,1,유흥,5,제주특별자치도 제주시 고마로17길 4,6_90%미만
288311,47에스티(47st),1,유흥,5,제주특별자치도 제주시 신대로14길 47,6_90%미만


In [10]:
# 요식데이터만 추출

# 요식/유흥 데이터 추출
jeju_restaurant = unique_df[unique_df['LG_CAT'] == 1]
# 유흥 데이터 제거
jeju_restaurant = jeju_restaurant[jeju_restaurant['MD_CAT'] != 5]

jeju_restaurant.sort_values('MNTH_SALES_PCTL')

,MCT_NM,LG_CAT,MD_CAT_NM,MD_CAT,MCT_BSE_AR,MNTH_SALES_PCTL
4805,주식회사리치모어,1,일식/중식/양식,2,제주특별자치도 제주시 공항로 2,1_상위10%이상
558,싱싱잇,1,한식,1,제주특별자치도 제주시 한림읍 한림로 181,1_상위10%이상
7284,카페노티드 제주애월,1,기타요식,4,제주특별자치도 제주시 애월읍 애월로1길 24-9,1_상위10%이상
1718,숙성도F&B,1,한식,1,제주특별자치도 제주시 노연로 48,1_상위10%이상
1778,가람돌솥밥,1,한식,1,제주 서귀포시 중문관광로 332,1_상위10%이상
...,...,...,...,...,...,...
237,고요새,1,일식/중식/양식,2,제주특별자치도 제주시 선사로8길 11,5_75%~90%
1830,금능반지하,1,일식/중식/양식,2,제주특별자치도 제주시 한림읍 금능9길 30,5_75%~90%
9026,메가엠지씨커피성산일출봉점,1,제과/커피/패스트푸드,3,제주특별자치도 서귀포시 성산읍 일출로 284-3,5_75%~90%
8573,메가엠지씨커피제주도청점,1,제과/커피/패스트푸드,3,제주특별자치도 제주시 신대로7길 3,5_75%~90%


In [11]:
# 매출 상위 10% 이상 매장
jeju_rest_10 = jeju_restaurant[jeju_restaurant['MNTH_SALES_PCTL'] == '1_상위10%이상']
jeju_rest_10.head()

,MCT_NM,LG_CAT,MD_CAT_NM,MD_CAT,MCT_BSE_AR,MNTH_SALES_PCTL
239,고이정,1,한식,1,제주특별자치도 제주시 애월읍 애월로 6,1_상위10%이상
312,대기정,1,한식,1,제주 서귀포시 이어도로 41,1_상위10%이상
319,델문도,1,기타요식,4,제주특별자치도 제주시 조천읍 조함해안로 519-10,1_상위10%이상
353,돈사돈,1,한식,1,제주 제주시 우평로 19,1_상위10%이상
365,돈이랑,1,한식,1,제주 서귀포시 일주서로 953,1_상위10%이상


In [12]:
jr_10_raw_addr = list(jeju_rest_10['MCT_BSE_AR'])

In [13]:
jr_10_name = list(jeju_rest_10['MCT_NM'])

In [14]:
location = list() # 위, 경도 저장
for item in jr_10_raw_addr:
    ret_location = getLatLng(item)
    location.append(ret_location)
location

[('33.4592977986301', '126.311352052723'),
 ('33.2402750470828', '126.428551115104'),
 ('33.5437022656737', '126.668709879591'),
 ('33.4788680102903', '126.464026131012'),
 ('33.2576980450082', '126.409094060753'),
 ('33.3863971803879', '126.227228353511'),
 ('33.50252795907', '126.513165064652'),
 ('33.2302341590121', '126.30881632585'),
 ('33.2392235100208', '126.319190550487'),
 ('33.5170715229989', '126.522082103902'),
 ('33.236084677694', '126.514923192083'),
 ('33.485803611054', '126.485125827282'),
 ('33.2496632565012', '126.431297158467'),
 ('33.5162575762232', '126.528057318421'),
 ('33.248515915462', '126.559286445208'),
 ('33.4794710006086', '126.47283548724'),
 ('33.2583337253269', '126.407510397505'),
 ('33.2177456029533', '126.249810398098'),
 ('33.3065311964764', '126.307543154776'),
 ('33.5059364682672', '126.495951277797'),
 ('33.3931778799612', '126.38317885864'),
 ('33.2412109273802', '126.378717518735'),
 ('33.5344013008299', '126.634132237491'),
 ('33.3902248567191

In [16]:
# 지도 생성
map = folium.Map(location=location[0], zoom_start=10)
map

In [17]:
# 마커 생성
folium.Marker(location=location[0], 
              popup=jr_10_name[0],
             icon=folium.Icon(color='pink', icon='glyphicon glyphicon-road')).add_to(map)
map

In [18]:
# 전체 마커 생성
for i in range(len(location)):
    folium.Marker(location=location[i], 
              popup=jr_10_name[i],
             icon=folium.Icon(color='blue', icon='glyphicon glyphicon-road')).add_to(map)
map

In [19]:
# 지도 중심 찾기
y_sum = x_sum = 0
y_mean = x_mean = 0
for item in location:
    y_sum += float(item[0])
    x_sum += float(item[1])
print(y_sum, x_sum)

x_mean = x_sum/len(location)
y_mean = y_sum/len(location)
print(y_mean, x_mean)

start_location = (y_mean, x_mean)

1870.1958657907817 7080.827504927904
33.39635474626396 126.443348302284


In [20]:
# 최종 지도출력
map = folium.Map(location=start_location, zoom_start=11)
url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query='
click_url = []

# for i in jr_10_name:
#     a = url + i.replace(' ','')
for i in range(len(location)):
    folium.Marker(location=location[i], 
              popup=(jr_10_name[i], url+jr_10_name[i].replace(' ','')),
             icon=folium.Icon(color='blue', icon='glyphicon glyphicon-road')).add_to(map)
map

In [75]:
map.save('jeju.html')

In [2]:
url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query='
click_url = []

In [15]:
jr_10_name

['고이정',
 '대기정',
 '델문도',
 '돈사돈',
 '돈이랑',
 '싱싱잇',
 '충민정',
 '남경미락',
 '원앤온리',
 '일통이반',
 '제스토리',
 '숙성도F&B',
 '가람돌솥밥',
 '고집돌우럭',
 '네거리식당',
 '늘봄흑돼지',
 '숙성도중문',
 '신미영이네',
 '신화돌담집',
 '제주향토골',
 '크라운돼지',
 '(주) 베이힐',
 '(주)산굼부리',
 '(주)한림공원',
 '제주색달식당',
 '제주오성본점',
 '주식회사새별',
 '칠돈가직영점',
 '숙성도 함덕점',
 '쌍둥이횟집본점',
 '자매국수삼도점',
 '제주광해애월점',
 '제주향토음식점',
 '우정회센타2호점',
 '주식회사 숙성도',
 '주식회사 싱싱잇',
 '명진전복주식회사',
 '주식회사리치모어',
 '(주)오설록 제주점',
 '고집돌우럭 중문점',
 '오가다 제주공항점',
 '주식회사 라포르엘',
 '주식회사 춘심이네',
 '(주)오드랑베이커리',
 '주식회사고등어쌈밥',
 '주식회사 우진해장국',
 '카페노티드 제주애월',
 '테디밸리골프앤리조트',
 '더클리프 (주)씨앤모어',
 '주식회사 바다를본돼지',
 '풀입채진고복식당제주공항점',
 '롯데리아 제주공항국내선탑승점',
 '파리바게뜨제주공항렌트카하우스점',
 '(주)파리크라상파리바게뜨 제주공항점',
 '(주)파리크라상파리바게뜨제주공항탑승점',
 '롯데지알에스(주)스카이31푸드에비뉴제주국제공항4층']

In [36]:
a=[]
for i in range(len(jr_10_name)):
    a.append(url + jr_10_name[i])
type(a[0])

str

In [ ]:
"<a href = \"" + str_link + "\" >" + str_link + "</a>"

In [74]:
# 최종 지도출력
map = folium.Map(location=start_location, zoom_start=11)
url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query='
click_url = []

# for i in jr_10_name:
#     a = url + i.replace(' ','')
for i in range(len(location)):
    folium.Marker(location=location[i], 
              popup="<a href = \"" + url + jr_10_name[i].replace(' ','') + "\" target=_blank>" + jr_10_name[i] + "</a>",
             icon=folium.Icon(color='blue', icon='glyphicon glyphicon-road')).add_to(map)
map

- 아이콘 클릭해서 나오는 링크 실행하고픔.. -> a href 태그 넣어서 해결